In [7]:
import sys
sys.path.append('utils/')

import pandas as pd
import datasets
from huggingface_hub import notebook_login
from utils.math_grader import grade_answer
from utils.qwen_math_parser import extract_answer
import re
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForCausalLM, GenerationConfig
from prompts import get_prompt_builder
from prompts.prompt_schemas import load_few_shot_prompts
from utils.eval_utils import RewardEvaluator
from functools import partial
from torch.utils.data import DataLoader
from trl.trainer.utils import batch_generation
import torch

In [9]:
config = {
    'model_path': 'Qwen/Qwen2.5-Math-1.5B-Instruct',
    'data_path': 'data/datasets/math500_level1',
    'cache_dir': '/home/data/v.moskvoretskii/cache/',
    'task_type': 'math',
    'few_shot_dir': 'few_shots',
    'question_col': 'problem',
    'gold_col': 'answer'
}


In [10]:
model = AutoModelForCausalLM.from_pretrained(
        config['model_path'],
        cache_dir=config['cache_dir'],
        torch_dtype=torch.bfloat16
    )
tokenizer = AutoTokenizer.from_pretrained(
        config['model_path'],
        use_fast=True,
        cache_dir=config['cache_dir']

    )

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [11]:
input_ids = torch.tensor([[151644,   8948,    198,   5501,   2874,   3019,    553,   3019,     11,                                                                                                                            
            323,   2182,    697,   1590,   4226,   2878,   1124,  79075,   2979,                                                                                                                            
           3417, 151645,    198, 151644,    872,    198,   9885,    279,  30232,                                                                                                                            
            315,    279,   1555,    400,     18,     87,     10,     20,     88,                                                                                                                            
             28,     17,     15,      3,    624, 151645,    198, 151644,  77091,                                                                                                                            
            198,   1249,   1477,    279,  30232,    315,    279,   1555,   2661,                                                                                                                            
            553,    279,  23606,  17767,     18,     87,    488,    220,     20,                                                                                                                            
             88,    284,    220,     17,     15,     59,    701,    582,   1184,                                                                                                                            
            311,  18130,    279,  23606,    304,    279,  30232,  44894,   1484,                                                                                                                            
           1352,     11,    892,    374,  17767,     88,    284,  14619,    488,                                                                                                                            
            293,     59,    568,    758,    419,   1352,     11,  17767,     76,                                                                                                                            
          57758,  10868,    279,  30232,    315,    279,   1555,    382,  10061,                                                                                                                            
            594,   1191,    448,    279,   2661,  23606,    510,  78045,     18,                                                                                                                            
             87,    488,    220,     20,     88,    284,    220,     17,     15,                                                                                                                            
             59,   2533,   1654,   1366,    311,  11625,    369,  17767,     88,                                                                                                                            
             59,    568,   5512,     11,  42123,    279,  17767,     88,     59,                                                                                                                            
           7287,   4991,    553,  32256,    287,  17767,     18,     87,  57758,                                                                                                                            
            504,   2176,  11067,    315,    279,  23606,    510,  78045,     20,                                                                                                                            
             88,    284,    481,     18,     87,    488,    220,     17,     15,                                                                                                                            
             59,   2533,   5847,     11,  21749,   1449,   4647,    553,    220,                                                                                                                            
             20,    311,  11625,    369,  17767,     88,     59,    982,  78045,                                                                                                                            
             88,    284,    481,     59,  37018,     90,     18,  15170,     20,                                                                                                                            
             92,     87,    488,    220,     19,     59,   2533,   7039,    279,                                                                                                                            
          23606,    374,    304,    279,  30232,  44894,   1484,   1352,  17767,                                                                                                                            
             88,    284,  14619,    488,    293,     59,    701,   1380,    279,                                                                                                                            
          35606,    315,  17767,     87,  57758,    374,    279,  30232,  17767,                                                                                                                            
             76,     59,    568,   5542,    279,  23606,  17767,     88,    284,                                                                                                                            
            481,     59,  37018,     90,     18,  15170,     20,     92,     87,                                                                                                                            
            488,    220,     19,     59,    701,    582,    646,   1490,    429,                                                                                                                            
            279,  30232,  17767,     76,  57758,    374,   1124,   4080,     59,                                                                                                                            
          37018,     90,     18,  15170,     20,  11035,   3593,  54815,     11,                                                                                                                            
            279,  30232,    315,    279,   1555,    374,   1124,  11520,  79075,                                                                                                                            
          19999,     59,  37018,     90,     18,  15170,     20,   3417,     59,                                                                                                                            
            568, 151645,    198, 151644,    872,    198,   3862,   2578,    387,                                                                                                                            
            458,   1465,    304,    279,   6291,   3403,   1576,    315,   6853,                                                                                                                            
            315,   8660,    315,    279,   3405,     13,   5209,   4396,    279,                                                                                                                            
           1465,     11,    421,    894,     11,    323,  18130,    279,   6291,                                                                                                                            
             13,  10224,    697,   1590,   4226,   2878,   1124,  79075,   2979,
           3417, 151645,    198, 151644,  77091,    198]]).to('cuda:0')

In [12]:
model = model.to('cuda:0')

In [13]:
output = model.generate(input_ids, max_new_tokens=20, top_p=0.8, temperature=0.7)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [14]:
tokenizer.batch_decode(output)

["<|im_start|>system\nPlease reason step by step, and put your final answer within \\boxed{{}}<|im_end|>\n<|im_start|>user\nFind the slope of the line $3x+5y=20$.\n<|im_end|>\n<|im_start|>assistant\nTo find the slope of the line given by the equation \\(3x + 5y = 20\\), we need to rewrite the equation in the slope-intercept form, which is \\(y = mx + b\\). In this form, \\(m\\) represents the slope of the line.\n\nLet's start with the given equation:\n\\[3x + 5y = 20\\]\n\nWe want to solve for \\(y\\). First, isolate the \\(y\\)-term by subtracting \\(3x\\) from both sides of the equation:\n\\[5y = -3x + 20\\]\n\nNext, divide every term by 5 to solve for \\(y\\):\n\\[y = -\\frac{3}{5}x + 4\\]\n\nNow the equation is in the slope-intercept form \\(y = mx + b\\), where the coefficient of \\(x\\) is the slope \\(m\\). From the equation \\(y = -\\frac{3}{5}x + 4\\), we can see that the slope \\(m\\) is \\(-\\frac{3}{5}\\).\n\nTherefore, the slope of the line is \\(\\boxed{-\\frac{3}{5}}\\).

In [4]:
def add_input_ids(example, prompt_func):
    input_ids = prompt_func(example)
    example['input_ids'] = input_ids
    return example

ds = datasets.load_from_disk(config['data_path'])

# 3) Initialize the tokenizer
print(f"[INFO] Loading tokenizer from: {config['model_path']}")
tokenizer = AutoTokenizer.from_pretrained(
    config['model_path'],
    use_fast=True,
    cache_dir=config['cache_dir']

)

# 4) Load the prompt builder + few-shot
prompt_builder = get_prompt_builder(config['task_type'])
initial_generation_few_shot = load_few_shot_prompts(
    config['few_shot_dir'],
    f"{config['task_type']}_initial"
)

initial_generation_prompt_func = partial(
    prompt_builder.build_initial_generation_prompt,
    tokenizer=tokenizer,
    question_col=config['question_col'],
    few_shot_prompts=initial_generation_few_shot,
    tokenize=True
)

ds = ds.map(partial(add_input_ids, prompt_func=initial_generation_prompt_func))

[INFO] Loading tokenizer from: Qwen/Qwen2.5-0.5B-Instruct
[INFO] No few-shot file found for prompt_type='math_initial' at 'few_shots/math_initial.json'. Using none.


In [64]:
ds['train'] = ds['train'].select_columns([config['question_col'], config['gold_col'], 'input_ids'])
ds['test'] = ds['test'].select_columns([config['question_col'], config['gold_col'], 'input_ids'])

In [72]:
collator = DataCollatorWithPadding(tokenizer)

def custom_collate_fn(features, config):
    # 1) Extract the text columns you want to keep
    #    but do NOT pass them to the default collator
    text_cols = [config['gold_col'], config['question_col']]
    batch_text = {}
    for col in text_cols:
        batch_text[col] = [f[col] for f in features]  # gather them as a list

    # 3) Use the default HF collator to pad the numeric parts only
    model_batch = collator([{'input_ids': f['input_ids']} for f in features])

    # 4) Attach the text lists back to the batch, as Python lists
    model_batch.update(batch_text)

    return model_batch

dataloader = DataLoader(
    ds['train'],
    batch_size=2,
    shuffle=False,
    collate_fn=partial(custom_collate_fn, config=config),
    drop_last=True,
)

In [73]:
for batch in dataloader:
    break

In [75]:
batch

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
            553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
             13, 151645,    198, 151644,    872,    198,   5501,   2874,   3019,
            553,   3019,     11,    323,   2182,    697,   1590,   4226,   2878,
           1124,  79075,   2979,   3417, 151645,    198, 151644,    872,    198,
             35,  24308,    279,  26313,    315,    220,     22,     16,    320,
           2593,    220,     18,   4292, 151645,    198, 151644,  77091,    198,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643],
        [151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
            5

In [9]:
model = AutoModelForCausalLM.from_pretrained(config['model_path'], cache_dir=config['cache_dir'])

In [10]:
init_generation_config = GenerationConfig(
            max_new_tokens=4,  # or some separate param, e.g. args.initial_answer_length
            temperature=1.0,
            top_k=0,
            top_p=1.0,
            do_sample=True,
            pad_token_id=None,
            eos_token_id=None,
        )

init_outputs, init_logits = batch_generation(
                        model,
                        batch['input_ids'],
                        2,
                        tokenizer.pad_token_id,
                        init_generation_config,
)

From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


In [11]:
context_len = batch['input_ids'].shape[1]
init_answers = init_outputs[:, context_len:]

In [12]:
tokenizer.batch_decode(init_answers, skip_special_tokens=True)

[' To determine the remainder', 'To determine whether the']

In [52]:
def build_correction_inputs_for_batch(
    batch,
    init_answer_tokens: torch.Tensor,
    tokenizer,
    prompt_builder,
    collator,
    question_col: str = "question",
    initial_answer_col: str = "initial_answer",
):
    """
    Given a batch of data (each row containing the question text), plus the
    model's initial answer tokens (init_answer_tokens) for each row,
    this function:

      1) Decodes each row's initial answer tokens -> string.
      2) Calls prompt_builder.build_correction_prompt(...) with that text,
         producing the final correction prompt text.
      3) Tokenizes that correction prompt text.

    Returns:
      A list of dicts, each containing:
         {
           "input_ids": ...,
           "attention_mask": ...,
           "prompt_text": ...,
           "question": ...,
           "init_answer_text": ...
         }
      or whatever structure you need for your next forward pass.
    """

    # init_answer_tokens shape is [batch_size, seq_len_of_initial]
    # We'll decode row-by-row
    init_answer_texts = tokenizer.batch_decode(init_answer_tokens, skip_special_tokens=True)

    # We will store the final "correction input" for each row
    batch_correction_inputs = []

    for i, init_ans_text in enumerate(init_answer_texts):
        question_text = batch[question_col][i]

        # Build a 'sample' dict as your prompt_builder expects:
        sample_for_prompt = {
            question_col: question_text,
            initial_answer_col: [init_ans_text],  # your builder uses a list for initial answers
        }

        # Now get the final correction prompt(s). Typically there's 1 prompt
        # in this scenario, but build_correction_prompt returns a list.
        corr_inputs = prompt_builder.build_correction_prompt(
            sample=sample_for_prompt,
            tokenizer=tokenizer,
            question_col=question_col,
            initial_answer_col=initial_answer_col,
            tokenize=True
        )
        # Since we used only 1 initial answer, corr_prompts[0] is the final text
        corr_inputs = corr_inputs[0]

        batch_correction_inputs.append({'input_ids': corr_inputs})


    collated_corrections = collator(batch_correction_inputs)
    batch['correction_input_ids'] = collated_corrections['input_ids']
    return batch


In [18]:
len(batch['input_ids'])

2

In [53]:
batch_correction_inputs = build_correction_inputs_for_batch(
    batch, 
    init_answers,
    tokenizer,
    prompt_builder,
    collator,
    question_col='problem',
)

In [54]:
batch

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
            553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
             13, 151645,    198, 151644,    872,    198,   5501,   2874,   3019,
            553,   3019,     11,    323,   2182,    697,   1590,   4226,   2878,
           1124,  79075,   2979,   3417, 151645,    198, 151644,    872,    198,
             35,  24308,    279,  26313,    315,    220,     22,     16,    320,
           2593,    220,     18,   4292, 151645,    198, 151644,  77091,    198,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643],
        [151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
            5

In [51]:
tokenizer.batch_decode(collated['input_ids'])

['<|im_start|>system\nPlease reason step by step, and put your final answer within \\boxed{{}}<|im_end|>\n<|im_start|>user\nDetermine the remainder of 71 (mod 3).\n<|im_end|>\n<|im_start|>assistant\n To determine the remainder<|im_end|>\n<|im_start|>user\nThere might be an error in the solution above because of lack of understanding of the question. Please correct the error, if any, and rewrite the solution. Put your final answer within \\boxed{{}}<|im_end|>\n<|im_start|>assistant\n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>',
 '<|im_start|>system\nPlease reason step b

In [45]:
processed[0]

{'input_ids': [151644,
  8948,
  198,
  5501,
  2874,
  3019,
  553,
  3019,
  11,
  323,
  2182,
  697,
  1590,
  4226,
  2878,
  1124,
  79075,
  2979,
  3417,
  151645,
  198,
  151644,
  872,
  198,
  35,
  24308,
  279,
  26313,
  315,
  220,
  22,
  16,
  320,
  2593,
  220,
  18,
  4292,
  151645,
  198,
  151644,
  77091,
  198,
  2014,
  8253,
  279,
  26313,
  151645,
  198,
  151644,
  872,
  198,
  3862,
  2578,
  387,
  458,
  1465,
  304,
  279,
  6291,
  3403,
  1576,
  315,
  6853,
  315,
  8660,
  315,
  279,
  3405,
  13,
  5209,
  4396,
  279,
  1465,
  11,
  421,
  894,
  11,
  323,
  18130,
  279,
  6291,
  13,
  10224,
  697,
  1590,
  4226,
  2878,
  1124,
  79075,
  2979,
  3417,
  151645,
  198,
  151644,
  77091,
  198]}

In [2]:
ds = datasets.load_from_disk('/home/data/v.moskvoretskii/cache/STaSC/test_math_qwen_promtping3/iter_0/data_test')

In [137]:
idx = 48

ans = ds['star_correction_initial_generation'][idx][0]
gt = ds['answer'][idx]


final = extract_answer(ans)
print(grade_answer(final, gt))

if not grade_answer(final, gt):
    print(ans)
    print(gt)


True


In [66]:
final = extract_answer(ans)

In [67]:
final

'(3,\\frac{\\pi}{2})'

In [68]:
final, gt

('(3,\\frac{\\pi}{2})', '\\left( 3, \\frac{\\pi}{2} \\right)')

In [69]:
grade_answer(final, gt)

True

In [3]:
df = pd.read_json('/home/data/v.moskvoretskii/prm800k/prm800k/math_splits/train.jsonl', lines=True)
train = datasets.Dataset.from_pandas(df)

In [4]:
df = pd.read_json('/home/data/v.moskvoretskii/prm800k/prm800k/math_splits/test.jsonl', lines=True)
test = datasets.Dataset.from_pandas(df)

In [5]:
math = datasets.DatasetDict({'train': train, 'test': test})

In [6]:
notebook_login()

Token has not been saved to git credential helper.


In [7]:
math.push_to_hub('VityaVitalich/MATH500')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/VityaVitalich/MATH500/commit/029807925a6c275a743315e3f806299706a87288', commit_message='Upload dataset', commit_description='', oid='029807925a6c275a743315e3f806299706a87288', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/VityaVitalich/MATH500', endpoint='https://huggingface.co', repo_type='dataset', repo_id='VityaVitalich/MATH500'), pr_revision=None, pr_num=None)

In [37]:
#math.save_to_disk('../data/math500')

Saving the dataset (1/1 shards): 100%|██████████| 500/500 [00:00<00:00, 45744.40 examples/s]


In [38]:
df = pd.read_json('/home/data/v.moskvoretskii/prm800k/prm800k/math_splits/train.jsonl', lines=True)
df = df[df['level'] == 1]
train = datasets.Dataset.from_pandas(df)

In [40]:
math = datasets.DatasetDict({'train': train, 'test': test})

In [41]:
math

DatasetDict({
    train: Dataset({
        features: ['problem', 'solution', 'answer', 'subject', 'level', 'unique_id', '__index_level_0__'],
        num_rows: 958
    })
    test: Dataset({
        features: ['problem', 'solution', 'answer', 'subject', 'level', 'unique_id'],
        num_rows: 500
    })
})

In [42]:
math.save_to_disk('../data/math500_level1')

Saving the dataset (1/1 shards): 100%|██████████| 500/500 [00:00<00:00, 89278.50 examples/s] 


In [19]:
df['level'].value_counts()

level
5    134
4    128
3    105
2     90
1     43
Name: count, dtype: int64

In [17]:
grade_answer(given_answer='14/3', ground_truth='\frac{14}{3}')

False

In [16]:
df['answer']

0      \left( 3, \frac{\pi}{2} \right)
1                                p - q
2                         \frac{14}{3}
3                                    9
4                        \text{Evelyn}
                    ...               
495               (2,12) \cup (12,102)
496                       \frac{5}{13}
497                        \frac{7}{2}
498                                 -1
499                          106^\circ
Name: answer, Length: 500, dtype: object